**CAPSTONE PROJECT**

with key usage
1. Agent
2. Agent Tool
3. Sessions & Memory

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
from pydantic import BaseModel, Field

# --- Tool 1: Pencatatan Laporan ---
class LaporanInput(BaseModel):
    """Skema input untuk mencatat laporan keluhan warga."""
    judul: str = Field(description="Judul singkat atau ringkasan keluhan.")
    deskripsi: str = Field(description="Detail lengkap dan lokasi kejadian keluhan.")
    kategori: str = Field(description="Kategori keluhan (ex: Infrastruktur, Lingkungan, Administrasi).")


def catat_laporan(judul: str, deskripsi: str, kategori: str) -> str:
    """Mencatat laporan baru ke dalam sistem basis data (simulasi)."""
    # Dalam implementasi nyata, fungsi ini akan memanggil API sistem backend pemerintah
    laporan_id = f"LAP-{hash(judul) % 10000}"
    print(f"\n[TOOL LOG: catat_laporan] Laporan telah dicatat.")
    print(f"ID: {laporan_id}, Kategori: {kategori}, Judul: {judul}")
    # Mengembalikan konfirmasi yang jelas kepada Agent
    return f"Laporan Anda (ID: {laporan_id}) berhasil dicatat dengan kategori {kategori}. Kami akan segera memprosesnya."

# --- Tool 2: Penentuan Unit Penanggung Jawab ---

def tentukan_unit_penanggung_jawab(kategori_laporan: str) -> str:
    """Menentukan dinas atau unit pemerintah yang bertanggung jawab berdasarkan kategori."""
    
    # 1. Normalisasi Input Agen: Ubah ke huruf kecil dan hapus spasi ekstra
    kategori_norm = kategori_laporan.lower().strip()
    
    # 2. Definisikan mapping dengan kunci huruf kecil
    mapping = {
        "infrastruktur jalan": "Dinas Pekerjaan Umum (DPU)",
        "lingkungan": "Dinas Lingkungan Hidup (DLH)",
        "administrasi": "Dinas Kependudukan dan Pencatatan Sipil (Dukcapil)",
        "administrasi kependudukan": "Bagian Pelayanan Terpadu Satu Pintu (PTSP)",
        "lainnya": "Sekretariat Daerah"
    }
    

    unit = mapping.get(kategori_norm, "Sekretariat Daerah")
    
    # Tambahkan logging untuk melihat apa yang sebenarnya dikirim Agent
    print(f"\n[TOOL LOG: tentukan_unit_penanggung_jawab] Kategori dari Agen: '{kategori_laporan}' -> Dicocokkan dengan: '{kategori_norm}'")
    
    return unit
def check_urgency_level(deskripsi_laporan: str) -> str:
    """Mengklasifikasikan urgensi laporan (Tinggi, Sedang, Rendah) berdasarkan kata kunci."""
    
    deskripsi_lower = deskripsi_laporan.lower()
    

    if any(keyword in deskripsi_lower for keyword in ["berbahaya", "ancaman", "segera", "darurat", "kebakaran", "ambruk", "kritis", "kecelakaan"]):
        urgensi = "Tinggi"
    elif any(keyword in deskripsi_lower for keyword in ["terlambat", "tidak berfungsi", "rusak", "mengganggu", "lama"]):
        urgensi = "Sedang"
    else:
        urgensi = "Rendah"

    print(f"[TOOL LOG: check_urgency_level] Urgensi ditentukan: {urgensi}")
    return urgensi

In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [5]:
async def run_session(
    runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"
):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [6]:
# Define constants used throughout the notebook
APP_NAME = "Agen_Pelaporan_dan_Keluhan_Warga"
USER_ID = "demo_user"

memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing


agent_tools = [catat_laporan, tentukan_unit_penanggung_jawab, check_urgency_level]

complaint_agent = LlmAgent(
    name="Agen_Pelaporan_dan_Keluhan_Warga",
    description="Agen AI untuk membantu warga mencatat keluhan, mengkategorikannya, dan mengarahkan ke unit penanggung jawab yang tepat.",
    tools=agent_tools,
    instruction="""
    Anda adalah Agen Pelaporan dan Keluhan Pemerintah yang profesional.
    Tugas Anda adalah:
    1. Ekstrak deskripsi laporan.
    2. Wajib MENGGUNAKAN TOOL 'check_urgency_level' untuk menentukan tingkat Urgensi (Tinggi, Sedang, Rendah).
    3. Wajib MENGGUNAKAN TOOL 'tentukan_unit_penanggung_jawab' untuk menentukan dinas terkait.
    4. Setelah memiliki Judul, Deskripsi, Kategori, dan Urgensi, Wajib MENGGUNAKAN TOOL 'catat_laporan'.
    5. Berikan konfirmasi yang ramah, Urgensi yang terdeteksi, dan ID laporan kepada pengguna.
    """,

    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config)
)



print("Agen Pelaporan dan Keluhan berhasil dibuat.")

Agen Pelaporan dan Keluhan berhasil dibuat.


In [7]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations
# Create runner with BOTH services
runner = Runner(
    agent=complaint_agent,
     app_name="Agen_Pelaporan_dan_Keluhan_Warga",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)


In [8]:
await run_session(
    runner,
    "Saya mau lapor, ada jalan berlubang besar di depan kantor walikota. Sangat berbahaya untuk pengendara motor!",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > Saya mau lapor, ada jalan berlubang besar di depan kantor walikota. Sangat berbahaya untuk pengendara motor!


[TOOL LOG: check_urgency_level] Urgensi ditentukan: Tinggi



[TOOL LOG: tentukan_unit_penanggung_jawab] Kategori dari Agen: 'Infrastruktur Jalan' -> Dicocokkan dengan: 'infrastruktur jalan'



[TOOL LOG: catat_laporan] Laporan telah dicatat.
ID: LAP-2077, Kategori: Infrastruktur Jalan, Judul: Jalan Berlubang di Depan Kantor Walikota


Model: > Terima kasih atas laporan Anda!
Keluhan Anda mengenai jalan berlubang di depan kantor walikota telah berhasil dicatat dengan Judul: **Jalan Berlubang di Depan Kantor Walikota**.
Tingkat urgensi laporan ini terdeteksi **Tinggi** karena potensi bahaya bagi pengendara.
Laporan Anda telah kami teruskan ke Dinas Pekerjaan Umum (DPU) dengan ID laporan: **LAP-2077**. Kami akan segera memprosesnya.


In [9]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)


print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")


await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")

📝 Session contains:
  user: Saya mau lapor, ada jalan berlubang besar di depan kantor walikota. Sangat berbahaya untuk pengendara motor!...
  model: None...
  user: None...
  model: None...
  user: None...
  model: None...
  user: None...
  model: Terima kasih atas laporan Anda!
Keluhan Anda mengenai jalan berlubang di depan kantor walikota telah berhasil dicatat dengan Judul: **Jalan Berlubang di Depan Kantor Walikota**.
Tingkat urgensi laporan ini terdeteksi **Tinggi** karena potensi bahaya bagi pengendara.
Laporan Anda telah kami teruskan ke Dinas Pekerjaan Umum (DPU) dengan ID laporan: **LAP-2077**. Kami akan segera memprosesnya....
✅ Session added to memory!


In [10]:
#test 2
await run_session(
    runner,
    "Saya ingin mengajukan komplain karena proses pembuatan akta kelahiran anak saya mengalami keterlambatan lebih dari batas waktu yang dijanjikan yaitu 14 hari jam kerja.",
    "conversation-02",  # Session ID
)



### Session: conversation-02

User > Saya ingin mengajukan komplain karena proses pembuatan akta kelahiran anak saya mengalami keterlambatan lebih dari batas waktu yang dijanjikan yaitu 14 hari jam kerja.


[TOOL LOG: check_urgency_level] Urgensi ditentukan: Sedang



[TOOL LOG: tentukan_unit_penanggung_jawab] Kategori dari Agen: 'Administrasi Kependudukan' -> Dicocokkan dengan: 'administrasi kependudukan'



[TOOL LOG: catat_laporan] Laporan telah dicatat.
ID: LAP-8223, Kategori: Administrasi Kependudukan, Judul: Keterlambatan Akta Kelahiran


Model: > Terima kasih atas laporan Anda. Laporan Anda mengenai "Keterlambatan Akta Kelahiran" dengan deskripsi "proses pembuatan akta kelahiran anak saya mengalami keterlambatan lebih dari batas waktu yang dijanjikan yaitu 14 hari jam kerja" telah berhasil kami catat.

Tingkat urgensi laporan Anda terdeteksi sebagai **Sedang**.
Nomor ID laporan Anda adalah: **LAP-8223**.

Unit yang bertanggung jawab atas laporan ini adalah Bagian Pelayanan Terpadu Satu Pintu (PTSP). Kami akan segera menindaklanjuti laporan Anda.


In [11]:

search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="apa nama unit penanggung jawab atas laporan ada lubang besar didepan gedung walikota?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 2 relevant memories

  [user]: Terima kasih atas laporan Anda!
Keluhan Anda mengenai jalan berlubang di depan kantor walikota telah berhasil dicatat dengan Judul: **Jalan Berlubang di Depan Kantor Walikota**.
Tingkat urgensi laporan ini terdeteksi **Tinggi** karena potensi bahaya bagi pengendara.
Laporan Anda telah kami teruskan ke Dinas Pekerjaan Umum (DPU) dengan ID laporan: **LAP-2077**. Kami akan segera memprosesnya....
  [Agen_Pelaporan_dan_Keluhan_Warga]: Terima kasih atas laporan Anda!
Keluhan Anda mengenai jalan berlubang di depan kantor walikota telah berhasil dicatat dengan Judul: **Jalan Berlubang di Depan Kantor Walikota**.
Tingkat urgensi laporan ini terdeteksi **Tinggi** karena potensi bahaya bagi pengendara.
Laporan Anda telah kami teruskan ke Dinas Pekerjaan Umum (DPU) dengan ID laporan: **LAP-2077**. Kami akan segera memprosesnya....
